In [1]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
persist_directory = "../database/vectorstore.db"

In [3]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct"
)

In [4]:
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_7092\2851879184.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [9]:
retriever = vectorstore.as_retriever()

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


template = """Answer the question based only on the following context:
{context}

Question: {query}
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = prompt | llm | StrOutputParser()

In [10]:
def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)


query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"

relevant_docs = retriever.invoke(query)
qa_chain.invoke({"context": format_docs(relevant_docs), "query": query})

'উত্তর: প্রশ্ন অনুযায়ী, প্রদত্ত প্রসঙ্গে “অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে”-এর সুনির্দিষ্ট উত্তর পাওয়া যায়নি।'